# The Product Pricer Continued

A model that can estimate how much something costs, from its description.

## Enter a Couple of Frontier Models:

And now - putting some Frontier Models to the test.

### 2 important points:

It's important to appreciate that any actual training on the frontier models won't be done in this notebook section. But what's done will be just providing them with the Test dataset to see how they perform. They don't gain the benefit of the 400,000 training examples that we provided to the Traditional ML models.

HAVING SAID THAT...

It's entirely possible that in their monstrously large training data, they've already been exposed to all the products in the training AND the test set. So there could be test "contamination" here which gives them an unfair advantage which is good to keep in mind.

In [3]:
import os 
import re
import math 
import json 
import random 
from dotenv import load_dotenv
from huggingface_hub import login 
import matplotlib.pyplot as plt 
import numpy as np 
import pickle
from collections import Counter

### Frontier models:
from openai import OpenAI 
from anthropic import Anthropic 

### Internal classes: 
from items import Item 
from testing import Tester 

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
### Environment 

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
HF_TOKEN = os.getenv("HF_TOKEN")

login(HF_TOKEN)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
openai_client = OpenAI() 
claude_client = Anthropic()

In [6]:
%matplotlib inline

In [7]:
### Load pickle files of datasets: 

with open("train.pkl", "rb") as f: 
    train = pickle.load(f)

with open("test.pkl", "rb") as f: 
    test = pickle.load(f)

## Human Prediction Model

In [8]:
### Write the test set to a CSV

import csv 

with open("human_input.csv", "w") as f: 
    writer = csv.writer(f)
    for t in test[:250]: 
        writer.writerow([t.test_prompt(), 0])

In [9]:
### Read it back in and manually fill in the price column with (human) price guesses
### And then, move each guesses price into human_predictions which will be used, aligned with 
### the human_pricer function (price generator function)

human_predictions = []

with open("human_output.csv", "r") as f:
    reader = csv.reader(f)

    for row in reader:
        human_predictions.append(float(row[1]))

In [10]:
def human_pricer(item): 
    idx = test.index(item)
    print("IDX: ", idx)
    return human_predictions[idx]

In [ ]:
Tester.test(human_pricer, test)

In [32]:
test[0].test_prompt().replace(" to the nearest dollar", "").replace("Price is $", "")

'How much does this cost?\n\nSydney Rustic Mirror - Vanity Mirror, Bathroom Mirror, Farmhouse Decor, Wood Mirror, Large Mirror - 4 Sizes & 20 Colors - Red Oak\nThe Sydney low profile thin wood framed mirror will blend effortlessly in your current decor creating a simple yet sophisticated look. Our mirror features a reclaimed rustic styled wood finish, strong decorative lines with a thin 2.25” inch wide frame maximizing the visible mirror. This hanging mirror can be mounted horizontally or vertically. Available in 20 Colors - Shown in Red Oak. Not sure on color we do color samples please contact us for details. Available in 4 sizes, 24x30, 36x30, 42x30 & 60x30, all measurements are overall dimensions including frame and mirror. We offer two types\n\n'

## GPT-4o-mini Prediction

In [11]:
# Strip out phrases such as:
# 1. to the nearest dollar
# 2. Price is $
### Reason: Now integrating with a GPT 5 model which is sophisticated enough 
### to handle it and understand without such guardrails 

def messages_for(item): 
    system_message = "You estimate the price of item. Reply only with the price without any additional explanation or comment."
    user_prompt = item.test_prompt().replace(" to the nearest dollar", "").replace("Price is $", "")

    return [
        {"role": "system", "content": system_message}, 
        {"role": "user", "content": user_prompt}, 
        {"role": "assistant", "content": "Price is $"} # little trick to encourage the response followed in this form
    ]

In [ ]:
messages_for(test[0])

In [12]:
### a utility function that extracts the price (float) 
### out of LLM's responses

def get_price(s: str): 
    s = s.replace("$", "").replace(",", "")
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [17]:
def gpt_5_pricer(item): 
    try:
        completion = openai_client.chat.completions.create(
            model="gpt-5-mini", 
            messages=messages_for(item)
        )
        response = completion.choices[0].message.content
        
        return get_price(response)
    except Exception as e: 
        print({e})
        continue

SyntaxError: 'continue' not properly in loop (4070876250.py, line 12)

In [18]:
Tester.test(gpt_5_pricer, test)

NameError: name 'gpt_5_pricer' is not defined

In [19]:
def claude_haiku_pricer(item): 
    messages = messages_for(item)
    system_message = messages[0]["content"]
    messages = messages[1:]
    completion = claude_client.messages.create(
        model="claude-3-5-haiku-latest", 
        max_tokens=5, 
        system=system_message, 
        messages=messages
    )
    response = completion.content[0].text 
    return get_price(response)

In [ ]:
Tester.test(claude_haiku_pricer, test)

In [21]:
Tester.test(gpt_5_pricer, test[0])

NameError: name 'gpt_5_pricer' is not defined

In [22]:
test[0].price

189.99